In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn import metrics
import tweepy

# 1. Run the following command in terminal to get the tweets_id

In [2]:
#reference: https://medium.com/@jcldinco/downloading-historical-tweets-using-tweet-ids-via-snscrape-and-tweepy-5f4ecbf19032
"""
snscrape twitter-search "#TSLA since:2020-04-01 until:2020-10-10" > TSLA.txt
snscrape twitter-search "#AAPL since:2020-04-01 until:2020-10-10" > AAPL.txt
snscrape twitter-search "#MSFT since:2020-04-01 until:2020-10-10" > MSFT.txt


snscrape twitter-search "#NIO since:2019-11-01 until:2020-11-01" > NIO.txt
snscrape twitter-search "#TSLA since:2019-11-01 until:2020-11-01" > TSLA.txt
snscrape twitter-search "#AAPL since:2019-11-01 until:2020-11-01" > AAPL.txt
snscrape twitter-search "#AAPL since:2019-11-01 until:2020-11-01" > MSFT.txt
"""

'\nsnscrape twitter-search "#TSLA since:2020-04-01 until:2020-10-10" > TSLA.txt\nsnscrape twitter-search "#AAPL since:2020-04-01 until:2020-10-10" > AAPL.txt\nsnscrape twitter-search "#MSFT since:2020-04-01 until:2020-10-10" > MSFT.txt\n\n\nsnscrape twitter-search "#NIO since:2019-11-01 until:2020-11-01" > NIO.txt\nsnscrape twitter-search "#TSLA since:2019-11-01 until:2020-11-01" > TSLA.txt\nsnscrape twitter-search "#AAPL since:2019-11-01 until:2020-11-01" > AAPL.txt\nsnscrape twitter-search "#AAPL since:2019-11-01 until:2020-11-01" > MSFT.txt\n'

# 2. Crawl tweets info based on the tweets_id

In [3]:
def tweetsCrawler(ticker):
    """
    crawl all the tweets in the specific period, and write them in ticker_tweets.csv
    """
    consumer_key = "aqL7hyXwXd2Czpn3GNIlWmXNc" 
    consumer_secret = "1JqdHCmidPQnYX4Mu3imGJEcQfpxyEh2z8K5WtgLQCtQ7SBxiQ" 
    access_token = "940903761915949057-OSZIlUYzerbIpXSUZQvYQxmusCSNdl0" 
    access_token_secret = "NMNWR5uNwYAx3QPXQefqDpbbvPbM6MyKRH5xfHuo5tr0E"
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    
    tweet_url = pd.read_csv(ticker+".txt", index_col= None, header = None, names = ["links"])
    #tweet_url.head()
    
    af = lambda x: x["links"].split("/")[-1]
    tweet_url['id'] = tweet_url.apply(af, axis=1)
    
    ids = tweet_url['id'].tolist()
    
    total_count = len(ids)
    chunks = (total_count - 1) // 50 + 1
    
    def fetch_tw(ids):
        list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
        empty_data = pd.DataFrame()
        for status in list_of_tw_status:
                tweet_elem = {"tweet_id": status.id,
                         "screen_name": status.user.screen_name,
                        "followers": status.user.followers_count,
                         "tweet":status.full_text,
                         "date":status.created_at}
                empty_data = empty_data.append(tweet_elem, ignore_index = True)
        empty_data.to_csv(ticker+"_tweets.csv", mode="a")
        
    for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        result = fetch_tw(batch)

# 3. Analyze each day's sentiment

In [4]:
def sentiment_score(ticker, weighted):
    """calculate each day's weighted sentiments, and save the results into a csv file
    """
    df = pd.read_csv(ticker+"_tweets.csv")
    df = df.dropna()    #remove the rows with na values
    df = df.reset_index()   # reset the index
    df = df.loc[:, ['date', 'followers', 'tweet']]    # only take the key info we'll use
    df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')    # transfer the type of date
    df['followers'] = df['followers'].astype(float)
    df['date'] = df['date'].dt.normalize()    # we only need the date
    
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    
    def sentiment_scores(sentence):
        """return a sentiment score of the input sentence
        """
        sid_obj = SentimentIntensityAnalyzer()
        sentiment_dict = sid_obj.polarity_scores(sentence)
        return sentiment_dict['compound']
    
    df['senti_score'] = 0 

    for i in range(0, len(df)):
        df.loc[i, 'senti_score'] = sentiment_scores(df.loc[i, 'tweet'])
        
    def weighted_avg(group, avg_name, weight_name):
        """ calculate weighted avg
        """
        d = group[avg_name]
        w = group[weight_name]
        try:
            return (d * w).sum() / w.sum()
        except ZeroDivisionError:
            return d.mean()
    
    if weighted == True:
        senti_series = df.groupby("date").apply(weighted_avg, "senti_score", "followers")
        filename = ticker+"_weighted_sentiment.csv"
        
    else:
        senti_series = df.groupby("date").mean().senti_score
        filename = ticker+"_avg_sentiment.csv"

    senti = pd.DataFrame({'tweets_date':senti_series.index, 'score':senti_series.values})
    senti['day'] = senti['tweets_date'].dt.dayofweek
    senti = senti.set_index("tweets_date")
    
    senti.to_csv(filename)

# 4. Join sentiment with stock movement

In [10]:
def data_processor(ticker, weighted):
    """join sentiment with stock movement
    """
    stock = pd.read_csv(ticker+"_stock.csv")
    ### true movement
    stock["Movement"] = stock["Adj Close"].pct_change() *100
    stock = stock.loc[:, ['Date', 'Movement']]
    stock['Date']=pd.to_datetime(stock['Date'], format='%Y-%m-%d') 
    
    import datetime
    stock.Date = stock.Date + datetime.timedelta(days = -1)
    stock = stock.set_index("Date")
    
    ####################
    ####join dataframes####
    ####################   
    if weighted == True:
        senti = pd.read_csv(ticker+"_weighted_sentiment.csv" , index_col=0)
    else:
        senti = pd.read_csv(ticker+"_avg_sentiment.csv", index_col=0)
    
    result = senti.join(stock, how="inner")
    result= result.dropna()
    if weighted == True:
        result.to_csv(ticker+"_weighted_joined.csv")
    else:
        result.to_csv(ticker+"_avg_joined.csv")

In [7]:
sentiment_score("MSFT", weighted=True)

In [11]:
data_processor("MSFT", weighted=True)

In [9]:
sentiment_score("MSFT", weighted=False)

In [12]:
data_processor("MSFT", weighted=False)

In [13]:
sentiment_score("AAPL", weighted=False)
data_processor("AAPL", weighted=False)

In [15]:
sentiment_score("AAPL", weighted=True)
data_processor("AAPL", weighted=True)

In [ ]:
def logisticRegression(ticker, weighted):
    if weighted == True:        
        df = pd.read_csv(ticker+'_weighted_joined.csv', index_col=0)
    else:
        df = pd.read_csv(ticker+'_avg_joined.csv', index_col=0)
    df['Direction'] = df.Movement.apply(lambda x: "up" if x>0 else "down")
    cvar = ["day", "Direction"]
    df[cvar] = df[cvar].astype('category')
    df = pd.get_dummies(df, prefix_sep='_')
    
    df2 = df.copy()
    rdummies = ["day_0", "Direction_down"]
    df2 = df2.drop(columns=rdummies)

    df3 = df2.copy()
    df3 = df3.drop(columns="Movement")
    
    testpart_size = 0.2
    df_nontestData, df_testData = train_test_split(df3, test_size=testpart_size, random_state=1)
    
    DV = 'Direction_up'
    y = df_nontestData[DV]
    X = df_nontestData.drop(columns=[DV])
    
    kfolds = 5
    min_alpha = 0.001
    max_alpha = 100
    n_candidates = 1000
    
    # C_list is the element-wise inverse of alpha_list. It is required as one of the paramater values for LogisticRegressionCV
    C_list = list(1/np.linspace(min_alpha, max_alpha, num=n_candidates))

    # Set n_jobs to be -1 to run LogisticRegressionCV on all CPU cores.
    clf_optimal = LogisticRegressionCV(Cs=C_list, cv=kfolds, penalty='l1', solver='saga', max_iter=5000, random_state=1, n_jobs=-1).fit(X,y)

        # y_test_actual is the actual values of the DV in the test partition
    y_test_actual = df_testData[DV]

    # X_test is the predictor values in the test partition
    X_test = df_testData.drop(columns=[DV])

    # Use predict method of the clf_optimal object to apply the model associated with clf_optimal to the test partition
    # y_test_predicted is the predicted values of the DV in the test partition 
    y_test_predicted = clf_optimal.predict(X_test)
    
    print(metrics.confusion_matrix(y_test_actual, y_test_predicted))
    print(clf_optimal.score(X_test, y_test_actual))

In [ ]:
data_processor("MSFT")

In [ ]:
logisticRegression("MSFT")

In [ ]:
#data_processor("AAPL")
logisticRegression("AAPL")

In [ ]:
#data_processor("TSLA")
logisticRegression("TSLA")

In [ ]:
#data_processor("NIO")
logisticRegression("NIO")

In [ ]:
data_processor("TSLA2",weighted=True)
logisticRegression("TSLA2",weighted=True)

In [ ]:
data_processor("TSLA2",weighted=False)
logisticRegression("TSLA2",weighted=False)

In [ ]:
data_processor("AAPL2",weighted=True)
logisticRegression("AAPL2",weighted=True)

In [ ]:
data_processor("AAPL2",weighted=0)
logisticRegression("AAPL2",weighted=0)